In [1]:
import numpy as np
import os
import json
import torch
import torch.nn as nn

In [126]:
# get real annotations
# get start frames of clips
# fine most common label for each clip
# new dataset yay
# fill into mask annotation and get loss???

In [2]:
# load train
train_file = '..\\task1_classic_classification\\task1_classic_classification\\calms21_task1_train.json'
with open(train_file, 'r') as file:
    train_data = json.load(file)['annotator-id_0']
# load test
test_file = '..\\task1_classic_classification\\task1_classic_classification\\calms21_task1_test.json'
with open(test_file, 'r') as file:
    test_data = json.load(file)['annotator-id_0']

In [3]:
def real_mask_annotations(data):
    real_annotations = []
    mask_annotations = []
    for video in data:
        real_annotations.append(np.array(data[video]['annotations']))
        arr = np.array(data[video]['annotations'])
        arr[arr < 2.5] = 0
        arr[arr > 2.5] = 1
        mask_annotations.append(arr)
    return real_annotations, mask_annotations
# annotations reduced to 1 and 0
real_annotations, mask_annotations = real_mask_annotations(train_data)
test_real_annotations, test_mask_annotations = real_mask_annotations(test_data)

In [4]:
def start_frames(data):
    train_data = data
    vid_start_frames = []
    curr_frame = 0
    for video_name in train_data.keys():
        vid_start_frames.append(curr_frame)
        vid_length = len(train_data[video_name]['annotations'])
        curr_frame += vid_length
    return vid_start_frames
# what frame does a video start at
vid_start_frames = start_frames(train_data)
test_vid_start_frames = start_frames(test_data)

In [5]:
# what frame of a video is an action at
def action_frame_to_video_frame(actions_frame, vid_start_frames):
    for i in range(len(vid_start_frames)-1):
        if actions_frame >= vid_start_frames[i] and actions_frame < vid_start_frames[i+1]:
            vid = i
            frame_in = (actions_frame - vid_start_frames[i])
            return vid, frame_in
    vid = len(vid_start_frames)-1
    frame_in = (actions_frame - vid_start_frames[-1])
    return vid, frame_in

In [6]:
def get_labels(file_path, real_annotations, vid_start_frames):
    train_preds = np.load(file_path)
    total = 0
    actions_frames = []
    for i in range(len(train_preds)-1):
        if train_preds[i] == 1 and train_preds[i+1] == 0:
            if np.unique(train_preds[i+1:i+300], return_counts=True)[1][0]/300 > 0.5:
                total += 1
                actions_frames.append(i+1)

    actual_labels = []
    for i in range(len(actions_frames)):
        vid_start_frame = actions_frames[i]
        vid, frame_in = action_frame_to_video_frame(vid_start_frame, vid_start_frames)
        if i == len(actions_frames) - 1:
            frame_size = 300
        else:
            frame_size = actions_frames[i+1] - vid_start_frame
        if frame_size >= 300:
            frame_size = 300
        a = np.argmax(np.bincount(real_annotations[vid][frame_in:frame_in+frame_size])[:3])
        # b = np.unique(real_annotations[vid][frame_in:frame_in+frame_size], return_counts=True)
        actual_labels.append(a)
        # value_pieces = real_annotations[vid][frame_in:frame_in+300]
    return actual_labels, actions_frames
    
actual_labels, actual_actions_frames = get_labels('real_train_pred.npy', real_annotations, vid_start_frames)
test_actual_labels, test_actions_frames = get_labels('test_files\\real_test_pred.npy', test_real_annotations, test_vid_start_frames)

In [7]:
from keras.utils import to_categorical
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [8]:
video_embeddings = torch.load('..\\train_task1_embeddings.pt', map_location=torch.device('cpu'))
test_video_embeddings = torch.load('..\\test_task1_embeddings.pt', map_location=torch.device('cpu'))

In [53]:
t2_video_embeddings = torch.load('..\\train_task2_embeddings.pt', map_location=torch.device('cpu'))
t2_test_video_embeddings = torch.load('..\\test_task2_embeddings.pt', map_location=torch.device('cpu'))

LETS JUST DO KNN

In [9]:
np.array(test_actual_labels)

array([1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 0, 2, 1, 2, 2, 2,
       2, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2,
       1, 1, 1, 2, 1, 1, 0, 0, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2,
       2, 2, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,

actual_labels.shape

In [54]:
t2_labels = [-1 for i in range(31)]
total_labels = actual_labels + t2_labels
a = np.array(video_embeddings).reshape(824,1536)
b = np.array(t2_video_embeddings).reshape(31,1536)
print(a.shape, b.shape)
total_x = np.concatenate((a,b))
print(total_x.shape)

(824, 1536) (31, 1536)
(855, 1536)


In [62]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
# neigh.fit(np.array(video_embeddings).reshape(824,1536), actual_labels)
neigh.fit(total_x, total_labels)



,n_neighbors,1
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [63]:
predictions = []
for i in range(len(test_video_embeddings)):
    video = test_video_embeddings[i]
    pred_val = neigh.predict(video)
    predictions.append(pred_val)
predictions = np.array(predictions)
test_labels = np.array(test_actual_labels)


In [64]:
len(t2_test_video_embeddings)

27

In [66]:
# FOR T2
predictions = []
for i in range(len(t2_test_video_embeddings)):
    video = t2_test_video_embeddings[i]
    pred_val = neigh.predict(video)
    predictions.append(pred_val)
predictions = np.array(predictions)
t2_test_labels = [-1 for i in range(27)]


In [67]:
f1_scores = sklearn.metrics.f1_score(t2_test_labels, predictions, average='macro')
rec_scores = sklearn.metrics.precision_score(t2_test_labels, predictions, average='macro')
prec_scores = sklearn.metrics.recall_score(t2_test_labels, predictions, average='macro')
print(f1_scores, rec_scores, prec_scores)

0.0 0.0 0.0


c:\Users\audre\anaconda3\envs\cmouse\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\audre\anaconda3\envs\cmouse\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [14]:
f1_scores = sklearn.metrics.f1_score(test_labels, predictions, average='macro')
rec_scores = sklearn.metrics.precision_score(test_labels, predictions, average='macro')
prec_scores = sklearn.metrics.recall_score(test_labels, predictions, average='macro')
print(f1_scores, rec_scores, prec_scores)

0.3287568924097426 0.37327415869095404 0.34371036508903297


In [15]:
accur = sklearn.metrics.accuracy_score(test_labels, predictions)
accur


0.4652777777777778

In [261]:
test_action_predictions = np.load('test_files\\real_test_pred.npy')

In [262]:
# make all 1 into 3 test_action_predictions
test_action_predictions[test_action_predictions==1] = 3
for i in range(len(test_actions_frames)):
    # fill in test_action_predictions
    start_fill = test_actions_frames[i]
    fill_with = predictions[i]
    test_action_predictions[start_fill + 1:][test_action_predictions[start_fill + 1:]!=3] = fill_with

In [263]:
np.unique(test_action_predictions, return_counts=True)

(array([0, 1, 2, 3]), array([ 21301,  82416,  10678, 147712]))

In [264]:
import itertools
official = np.array(list(itertools.chain.from_iterable(test_real_annotations)))
ours = test_action_predictions
accur = sklearn.metrics.accuracy_score(official, ours)
f1_scores = sklearn.metrics.f1_score(official, ours, average='macro')
rec_scores = sklearn.metrics.precision_score(official, ours, average='macro')
prec_scores = sklearn.metrics.recall_score(official, ours, average='macro')
print(accur, f1_scores, rec_scores, prec_scores)

0.7273975895340452 0.4556433034793878 0.4940225088151319 0.4631689504164316


In [118]:
# Convert to PyTorch Tensors
X_tensor = torch.cat(video_embeddings, dim=0)
y_tensor = torch.tensor(to_categorical(torch.reshape(torch.tensor(actual_labels), (len(actual_labels), 1))))
X_test = torch.cat(test_video_embeddings, dim=0)
y_test = torch.tensor(to_categorical(torch.reshape(torch.tensor(test_actual_labels), (len(test_actual_labels), 1))))

# 2. Define the Neural Network
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = torch.sigmoid(out)
        return out

# 3. Instantiate the Model, Loss Function, and Optimizer
input_size = video_embeddings[0].shape[1]
hidden_size = 10
output_size = 3
learning_rate = 0.00000001
num_epochs = 200
batch_size = 1
batch_no = len(actual_labels) // batch_size

model = SimpleNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [119]:
for epoch in range(num_epochs):

    # x_train, y_train = shuffle(x_train, y_train)
    x_train, y_train = X_tensor, y_tensor
    # Mini batch learning
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size

        x_var = x_train[start:end]
        y_var = y_train[start:end]
        # print(y_var)

        # Restart the gradients
        optimizer.zero_grad()

        # Run a training step: Pass the training data to
        # the neural network layers
        ypred_var = model(x_var)
        # print(ypred_var)

        # Backward and optimize
        loss = criterion(ypred_var, y_var)
        loss.backward()
        optimizer.step()
        # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    if (epoch + 1) % 1 == 0:
        # other_crit = nn.CrossEntropyLoss()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}') #, Test Loss: {other_crit(model(X_test), y_test)}')
        print('train',np.sum(0==(np.argmax(model(X_tensor).detach().numpy(), axis=1) - np.argmax(y_tensor.detach().numpy(),axis=1))))
        print('test',np.sum(0==(np.argmax(model(X_test).detach().numpy(), axis=1) - np.argmax(y_test.detach().numpy(),axis=1))))
    # # Forward pass
    # outputs = model(X_tensor)
    # loss = criterion(outputs, y_tensor)

    # # Backward and optimize
    # optimizer.zero_grad()
    # loss.backward()
    # optimizer.step()

    # if (epoch + 1) % 100 == 0:
    #     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# # 5. Evaluation
# with torch.no_grad():
#     predicted = (torch.sigmoid(model(X_tensor)) > 0.5).float()
#     accuracy = (predicted == y_tensor).sum().item() / y_tensor.size(0)
#     print(f'Accuracy of the model on the synthetic data: {accuracy:.2f}')

Epoch [1/200], Loss: 1.0714
train 101
test 76
Epoch [2/200], Loss: 1.0729
train 101
test 76
Epoch [3/200], Loss: 1.0744
train 101
test 76
Epoch [4/200], Loss: 1.0760
train 101
test 76
Epoch [5/200], Loss: 1.0775
train 101
test 76
Epoch [6/200], Loss: 1.0790
train 101
test 76
Epoch [7/200], Loss: 1.0805
train 101
test 76
Epoch [8/200], Loss: 1.0820
train 101
test 76
Epoch [9/200], Loss: 1.0835
train 101
test 76
Epoch [10/200], Loss: 1.0850
train 101
test 76
Epoch [11/200], Loss: 1.0866
train 101
test 76
Epoch [12/200], Loss: 1.0881
train 101
test 76
Epoch [13/200], Loss: 1.0896
train 101
test 76
Epoch [14/200], Loss: 1.0911
train 101
test 76
Epoch [15/200], Loss: 1.0926
train 101
test 76
Epoch [16/200], Loss: 1.0941
train 101
test 76
Epoch [17/200], Loss: 1.0956
train 101
test 76
Epoch [18/200], Loss: 1.0971
train 101
test 76
Epoch [19/200], Loss: 1.0986
train 101
test 76
Epoch [20/200], Loss: 1.1001
train 101
test 76
Epoch [21/200], Loss: 1.1016
train 101
test 76
Epoch [22/200], Loss: 

KeyboardInterrupt: 

In [ ]:
test_embeddings = torch.load('..\\test_task1_embeddings.pt', map_location=torch.device('cpu'))
X_tensor = torch.cat(test_embeddings, dim=0)
y_tensor = torch.tensor(to_categorical(torch.reshape(torch.tensor(actual_labels), (len(actual_labels), 1))))
test_loss = criterion(model(x_var), y_var)

In [ ]:
a = np.argmax(model(X_tensor).detach().numpy(), axis=1)
b = np.argmax(y_tensor, axis=1)

In [ ]:
np.equal(a,b)
np.unique(np.equal(a,b), return_counts=True)

C:\Users\audre\AppData\Local\Temp\ipykernel_36096\1689630220.py:1: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  np.equal(a,b)
C:\Users\audre\AppData\Local\Temp\ipykernel_36096\1689630220.py:2: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  np.unique(np.equal(a,b), return_counts=True)


(array([0, 1], dtype=uint8), array([249, 575]))

In [ ]:
test_embeddings = torch.load('..\\test_task1_embeddings.pt', map_location=torch.device('cpu'))

In [13]:
import sklearn

In [ ]:
f1_scores = sklearn.metrics.f1_score(labels, predictions, average=None)
rec_scores = sklearn.metrics.precision_score(labels, predictions, average=None)
prec_scores = sklearn.metrics.recall_score(labels, predictions, average=None)

# Average precsion - all labels not equal to correct label are mistakes
ap_scores = []
for single_label in sorted(np.unique(labels)):
    labels_l = labels == single_label
    probabilites_l = probabilites[:, single_label]
    ap_score_l = sklearn.metrics.average_precision_score(
        labels_l, probabilites_l, average="micro"
    )
    ap_scores.append(ap_score_l)

classes = sorted(self.class_to_number, key=self.class_to_number.get)
metrics = pd.DataFrame(
    {
        "Class": classes,
        "F1": f1_scores,
        "Precision": prec_scores,
        "Recall": rec_scores,
        "Average Precision": ap_scores,
    }
)